In [17]:
# import Stemmer
import re
import bisect
import xml.sax
import timeit
import subprocess
import mwparserfromhell
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import Stemmer
import re
import math
import timeit
stop_words = set(stopwords.words('english'))
stemmer = Stemmer.Stemmer('english')

reg1 = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',re.DOTALL)
reg2 = re.compile(r'{\|(.*?)\|}',re.DOTALL)
reg3 = re.compile(r'{{v?cite(.*?)}}',re.DOTALL)
refreg=r'{{v?cite(.*?)}}'
reg4 = re.compile(r'[-.,:;_?()"/\']',re.DOTALL)
reg5 = re.compile(r'\[\[file:(.*?)\]\]',re.DOTALL)
# reg6 = re.compile(r'[\'~` \n\"_!=@#$%-^*+{\[}\]\|\\<>/?]',re.DOTALL)
reg6 = re.compile(r'[\'~`\"_!=@#$%\-^*+{\[}\]\|\\<>/?]',re.DOTALL)

catRegExp = r'\[\[category:(.*?)\]\]'
infoRegExp = r'{{infobox(.*?)}}'
refRegExp = r'== ?references ?==(.*?)=='
reg7 = re.compile(infoRegExp,re.DOTALL)
reg8 = re.compile(refRegExp,re.DOTALL)
reg9 = re.compile(r'{{(.*?)}}',re.DOTALL)
reg10 = re.compile(r'<(.*?)>',re.DOTALL)
reg11=re.compile(r'\[\[(.*?)\]\]')
reg12=re.compile(catRegExp,re.DOTALL)

index_path="inverted_index/merged_path/"

def gettitles(toplist):
    top_titles=[]
    for docid in toplist:
        title_file_no=math.floor(docid/30000)
        title_index=docid%30000
        fp=open("title/"+str(title_file_no),'r')
        
        i=0
        for line in fp.readlines():
            if(i==title_index):
                top_titles.append(line[:-1])
            i+=1
    return top_titles
        
        
        
        
    

 
def convert(titledict):
    resdict={}
    for key,val in titledict.items():
        val=val.strip()
        token=val.split()
        token=stemmer.stemWords(token)
        for t in token:
           
            resdict[t]=key
    return resdict

def getposting(index_path,word):
#     print(word)
    secondaryindex_fp=open(index_path+"secondary_index.txt",'r')
    secondary_list=[]
    for line in secondaryindex_fp.readlines():
        secondary_list.append(line[:-1])
        
    index_no=bisect.bisect(secondary_list, word)-1 #index number fetched from secondary index
    if index_no==-1:                  #invalid index number
        return ""
    index_path=index_path+str(index_no)+".txt"   # path to index file to be searched
    index_fp=open(index_path,'r')
    
    for entry in index_fp.readlines():
        idx=entry.index('#')
        w=entry[:idx]
        if(w==word):
            return entry[idx:]
    return ""                                   # if posting doesn't exist return empty string

def gettopten(docid_freq_tfidf,top_k):
    rank_list=[]
    top_ten=[]
    
    for docid in docid_freq_tfidf:
#         print(docid)
        rank_list.append([[docid_freq_tfidf[docid][0],docid_freq_tfidf[docid][1]],docid])
        
    rank_list.sort(reverse=True)              # if number of query tokens are same in doc1 and doc2 then go for tfidf           
    
    i=0
    for item in rank_list:
        top_ten.append(item[1])
        i+=1
        if i == top_k:
            break
            
    return top_ten
        
        
        
        
        
        
    
    
   
    
        
        
    


# index_path="inverted_index/merged_path/"

# index_path="inverted_index/merged_path/"



def top_search1(st,top_k):

    st=st.lower()
    index=st.find(':')
    docid_freq_tfidf={}  # dict keys are doc_id and value is list of frequescy and tfidf weight
                         # where frequency is no of tokens from query are present in doc 
                         # and tfidf weight is the sum of all tfidfs of word
    if(index>=0):
        titledict={}

        field='#' 
        buffer=""
        for i in range(len(st)):
            if st[i]==':':
                if field!='#':
                    titledict[field]=buffer[0:len(buffer)-1]
                    buffer=""
                field=st[i-1]
                buffer=""

            else:
                buffer=buffer+st[i]

        if field!='#':
            titledict[field]=buffer
        resdict=convert(titledict)     #dict of word and their field type
#         print(resdict)


        for word,field in resdict.items():
#             print(word,field,"#####################################################################################################################################################################################################################################################3")
            postings=getposting(index_path,word)
    #         print(postings)

            if postings=="":
                continue


            postings=postings[:-1]  #remove new line

            postings=postings.split('#')
            postings=postings[1:]   #first ele in postings is always "" so remove it
    #         print(postings)

            for posting in postings:
            
            
                found=False
                if field=='c':
                    postinglist=re.findall(r'c[0-9]*',posting,re.DOTALL)
                    if(len(postinglist)):
                        tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                        found=True
                        
                if field=='i':
                    postinglist=re.findall(r'i[0-9]+',posting,re.DOTALL)
                    if(len(postinglist)):
                        tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                        found=True
                    
                if field=='t':
                    postinglist=re.findall(r't[0-9]*',posting,re.DOTALL)
                    if(len(postinglist)):
                        tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                        found=True
                if field=='e':
                    postinglist=re.findall(r'e[0-9]*',posting,re.DOTALL)
                    if(len(postinglist)):
                        tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                        found=True
                if field=='b':
                    postinglist=re.findall(r'b[0-9]*',posting,re.DOTALL)
                    if(len(postinglist)):
                        tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                        found=True
                if field=='r':
                    postinglist=re.findall(r'r[0-9]*',posting,re.DOTALL)
                    if(len(postinglist)):
                        tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                        found=True
                if(found==False):     #field not found so overall tfidf returned
                    poslist=posting.split('@')
                    tfidf=int(poslist[1])
                    prepost=poslist[0]

                    docid=re.findall(r'id[0-9]*',posting,re.DOTALL)

                    docid=int(re.findall(r'[0-9]+',docid[0],re.DOTALL)[0])

                    if docid not in docid_freq_tfidf:
                        fre_tfidfsum=[1,tfidf]
                        docid_freq_tfidf[docid]=fre_tfidfsum
                    else:
                        fre_tfidfsum=docid_freq_tfidf[docid]
                        fre_tfidfsum[0]+=1
                        fre_tfidfsum[1]+=tfidf
                        docid_freq_tfidf[docid]=fre_tfidfsum
                else:   #field found so field's tfidf considered
#                     poslist=posting.split('@')
#                     tfidf=int(poslist[1])
#                     prepost=poslist[0]

                    docid=re.findall(r'id[0-9]*',posting,re.DOTALL)

                    docid=int(re.findall(r'[0-9]+',docid[0],re.DOTALL)[0])

                    if docid not in docid_freq_tfidf:
                        fre_tfidfsum=[1,tfidf]
                        docid_freq_tfidf[docid]=fre_tfidfsum
                    else:
                        fre_tfidfsum=docid_freq_tfidf[docid]
                        fre_tfidfsum[0]+=1
                        fre_tfidfsum[1]+=tfidf
                        docid_freq_tfidf[docid]=fre_tfidfsum




        top_ten=gettopten(docid_freq_tfidf,top_k) #now working for to k instead of top 10.
        top_titles=gettitles(top_ten)
#         print(top_ten)
#         print(top_titles)

    else:
        st=reg1.sub(' ',st)
        st=reg2.sub(' ',st)
        st=reg3.sub(' ',st)
        st=reg4.sub(' ',st)
        st=reg5.sub(' ',st)
        st=reg6.sub(' ',st)
        st=st.strip()
        st=st.split()

        st=[word for word in st if word not in stop_words]
        st=stemmer.stemWords(st)

        for word in st:
#             print(word)
            postings=getposting(index_path,word)
    #         if word.isnumeric():
    #             print(postings)
            if postings=="":
                continue


            postings=postings[:-1]  #remove new line

            postings=postings.split('#')
            postings=postings[1:] 

            for posting in postings:



#                 poslist=posting.split('@')
#                 tfidf=int(poslist[1])-10
#                 prepost=poslist[0]
                
                #precedence order  title>infobox=category>ref=external link>body
                if(len(re.findall(r't[0-9]*',posting,re.DOTALL))):
                    postinglist=re.findall(r't[0-9]*',posting,re.DOTALL)
                    tfidf=tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                    docid=re.findall(r'id[0-9]*',posting,re.DOTALL)
                    docid=int(re.findall(r'[0-9]+',docid[0],re.DOTALL)[0])
                elif(len(re.findall(r'i[0-9]+',posting,re.DOTALL))):
                    postinglist=re.findall(r'i[0-9]+',posting,re.DOTALL)
                    tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                    docid=re.findall(r'id[0-9]*',posting,re.DOTALL)
                    docid=int(re.findall(r'[0-9]+',docid[0],re.DOTALL)[0])
                elif(len(re.findall(r'c[0-9]*',posting,re.DOTALL))):
                    postinglist=re.findall(r'c[0-9]*',posting,re.DOTALL)
                    tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                    docid=re.findall(r'id[0-9]*',posting,re.DOTALL)
                    docid=int(re.findall(r'[0-9]+',docid[0],re.DOTALL)[0])
                elif(len(re.findall(r'r[0-9]*',posting,re.DOTALL))):
                    postinglist=re.findall(r'r[0-9]*',posting,re.DOTALL)
                    tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                    docid=re.findall(r'id[0-9]*',posting,re.DOTALL)
                    docid=int(re.findall(r'[0-9]+',docid[0],re.DOTALL)[0])
                elif(len(re.findall(r'e[0-9]*',posting,re.DOTALL))):
                    postinglist=re.findall(r'e[0-9]*',posting,re.DOTALL)
                    tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                    docid=re.findall(r'id[0-9]*',posting,re.DOTALL)
                    docid=int(re.findall(r'[0-9]+',docid[0],re.DOTALL)[0])
                elif(len(re.findall(r'b[0-9]*',posting,re.DOTALL))):
                    postinglist=re.findall(r'b[0-9]*',posting,re.DOTALL)
                    tfidf=int(re.findall(r'[0-9]+',postinglist[0],re.DOTALL)[0])
                    docid=re.findall(r'id[0-9]*',posting,re.DOTALL)
                    docid=int(re.findall(r'[0-9]+',docid[0],re.DOTALL)[0])
                


#                 docid=re.findall(r'id[0-9]*',posting,re.DOTALL)
#                 docid=int(re.findall(r'[0-9]+',docid[0],re.DOTALL)[0])

                if docid not in docid_freq_tfidf:
                    fre_tfidfsum=[1,tfidf]
                    docid_freq_tfidf[docid]=fre_tfidfsum
                else:
                    fre_tfidfsum=docid_freq_tfidf[docid]
                    fre_tfidfsum[0]+=1
                    fre_tfidfsum[1]+=tfidf
                    docid_freq_tfidf[docid]=fre_tfidfsum

        top_ten=gettopten(docid_freq_tfidf,top_k)   #working for top k now
 
        top_titles=gettitles(top_ten)
        
        
        
    return (top_ten,top_titles)
            
            
def main():

    query_in=open("queries.txt",'r')
    query_out=open("queries_op.txt",'w')

    for line in query_in.readlines():
        print(line)
        line=line.split(',')
        top_k=int(line[0])
        query=line[1][:-1]

        start = timeit.default_timer()
        ids_titles=top_search1(query,top_k)         #main search logic return tuple of ids and titles list
        print(ids_titles,"##########################################################3")
        end = timeit.default_timer()

        for n in range(top_k):
            if n >= len(ids_titles[0]):       #if ids_titles doesn't have the number of documents we are asking for
                break
            write_line=str(ids_titles[0][n])+","+str(ids_titles[1][n])+"\n"
            query_out.write(write_line)

        t=end-start
        query_out.write(str(round(t,3))+","+str(round(t/top_k,3))+"\n")
        query_out.write("\n")

    query_out.close()
    print("End of processing")

if __name__ == "__main__": 
	main() 
        
        
#     query_out.write("\n")
    
        
    
    

           

                    
                    
                    
                    
                    
                    
    
                    
                    
                    
                
                
                
                
                
                
                
                
                
        
            
                
            
            
            
        
        
        
            
            
        
    

        

# else:
#     st=st.strip()
#     tokens=st.split()
#     tokens=stemmer.stemWords(tokens)
#     fp=open('demo.txt','r')
#     lines=fp.readlines()
#     for line in lines:
#         idx=line.find('#')
#         word=line[0:idx]
#         if word in tokens:
#             print(line)
            

        




            

10, b:Taj mahal, i:1999

([4271407, 4504998, 1905557, 1573248, 7865321, 1789027, 8449929, 8187833, 7810043, 8729462], ['Manoj Bharathiraja', 'Tourism in Uttar Pradesh', 'List of sultans of the Maldives', 'Mohammed Uthman al-Mirghani al-Khatim', 'Gateway of India', 'Eastern journey of Nicholas II', 'Mumbai culture', 'List of state leaders in 1415', 'Gujranwala', 'Duke Erikson']) ##########################################################3
10, t:russia relation

([7180136, 5576419, 4796581, 6433295, 6433293, 6433292, 6433291, 4787904, 4488269, 4332535], ['US - Russia Relations', 'NZ-Russia Relations', 'Russia-NATO relations', 'UK - Russia relations', 'UK-Russia relations', 'Russia - UK relations', 'Russia-UK relations', 'NATO-Russia relations', 'DPRK-Russia relations', 'Russia-U.S. relations']) ##########################################################3
10, i:Andrey Savin, b:Speedway World Cup

([2144069, 701911, 6783434, 5509479, 7160587, 7492305, 3792764, 7070899, 5554788, 4365229], ['L

([8650993, 8612804, 7986477, 7986441, 7843327, 7355697, 5011555, 4346777, 4336157, 4336086, 3205982, 3059439, 1616272, 1097979, 775230, 617913, 2148, 8091517, 480165, 8621759], ['Blade Runner (themes)', 'Blade Runner (soundtrack)', 'Blade Runner(computer game)', 'Blade Runner (computer game)', 'Blade Runner (1997 video game)', 'Blade Runner soundtrack', 'Blade Runner (soundtracks)', 'Blade Runner (1985 videogame)', 'Blade Runner (1985 video game)', 'Blade Runner (video game)', 'Blade Runner soundtracks', 'Blade Runner (versions)', 'Blade Runner (game)', 'Blade Runner (disambiguation)', 'Blade Runner (videogame)', 'Blade Runner (film)', 'Blade Runner', 'Blade runner', 'T-Blades', 'Themes in Blade Runner']) ##########################################################3
5, v for vendetta

([5036069, 4035925, 3608703, 1438542, 8587664], ['File:Portman V.jpg', 'File:Norsefire-logo.png', 'File:250px-Sutler2.jpg', 'File:Adam Susan.jpg', 'V for Vendetta (film)']) #################################